In [13]:
from numba import cuda
from graphcuda import generate_random_adjacency_matrix, faq_align, faq_align_perm
import time, math

In [14]:
@cuda.jit
def permute_graph(indices, input_graph, output_graph):
    x, y = cuda.grid(2)
    size = indices.shape[0]

    if x >= size or y >= size:
        return
    
    # Swap row
    output_graph[indices[y], indices[x]] = input_graph[y, x]

In [20]:
n_nodes = 2000
prob_edge = 1.2/(100-1)

A = generate_random_adjacency_matrix(n_nodes, prob_edge)
B = generate_random_adjacency_matrix(n_nodes, prob_edge)

In [21]:
block_dim = 16
grid_dim = math.ceil(A.shape[0] / block_dim)

ori_g = cuda.to_device(B)
new_g = cuda.device_array_like(ori_g)

start = time.perf_counter()
perm = faq_align_perm(A, B)
permute_graph[(grid_dim, grid_dim), (block_dim, block_dim)](perm, ori_g, new_g)
end = time.perf_counter()

end - start

24.170089200000007

In [22]:
start = time.perf_counter()
B_aligned = faq_align(A, B)
end = time.perf_counter()
end - start

24.374855199999956